In [1]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "BAAI/bge-reranker-v2-m3", cache_dir="rerank_model"
)
model = AutoModelForSequenceClassification.from_pretrained(
    "BAAI/bge-reranker-v2-m3", cache_dir="rerank_model"
)
model.eval()

pairs = [
    ["what is panda?", "hi"],
    [
        "what is panda?",
        "The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.",
    ],
]
with torch.no_grad():
    inputs = tokenizer(
        pairs, padding=True, truncation=True, return_tensors="pt", max_length=512
    )
    scores = (
        model(**inputs, return_dict=True)
        .logits.view(
            -1,
        )
        .float()
    )
    print(scores)

tensor([-8.1838,  5.2650])
